In [1]:
import torch
# import torch.nn as nn
import torch.optim as optim
import os
import copy
import json
import argparse
import datetime
import collections
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle

## input

特征：回看过去30天的6个量价特征，未做 RobustZScoreNorm 处理

In [2]:
c0 = pd.read_feather('data2/kbase6/close_2007-01.feather')
c0.head()

,end_date,stock_code,close_day_lag0,close_day_lag1,close_day_lag2,close_day_lag3,close_day_lag4,close_day_lag5,close_day_lag6,close_day_lag7,...,close_day_lag20,close_day_lag21,close_day_lag22,close_day_lag23,close_day_lag24,close_day_lag25,close_day_lag26,close_day_lag27,close_day_lag28,close_day_lag29
0,2007-01-04,000001.SZ,0.0,1.152984,-0.189501,-0.707711,-1.270391,-0.974698,-1.287721,-1.019862,...,-0.542033,-0.956269,-0.796646,-0.933699,-0.940420,-0.778582,-0.708488,-0.709334,-0.576758,-1.092390
1,2007-01-04,000002.SZ,0.0,-0.116772,-0.825266,-0.430052,-0.539504,-0.561118,-1.013698,-0.928439,...,-1.665469,-1.780526,-1.797305,-2.314282,-2.285733,-2.222334,-2.215843,-2.007260,-2.038216,-1.916342
2,2007-01-04,000004.SZ,0.0,0.445227,-0.439058,0.644170,0.652227,1.248003,1.261231,0.882330,...,0.168559,0.107048,0.228614,0.127827,-0.238106,-0.137482,-0.225478,-0.291959,-0.409587,-0.330771
3,2007-01-04,000005.SZ,0.0,0.365916,0.000000,0.673806,0.824679,1.025687,0.535000,0.510086,...,0.284925,0.079114,0.179871,-0.015252,-0.044395,-0.196832,-0.160935,-0.245473,-0.268632,-0.090059
4,2007-01-04,000006.SZ,0.0,-0.956589,-1.127663,-0.208254,-0.460917,0.105670,-0.566672,0.102743,...,-0.459295,-0.571200,-0.607484,-0.875841,-0.901658,-1.194287,-1.108976,-1.167553,-1.144199,-1.468312


收盘价：用于计算收益率，后作 CSRankNorm 成为label

In [3]:
df_close = pd.read_pickle('data2/EODPrices_adjclose.pkl')
df_close.head()

,s_info_windcode,trade_dt,s_dq_adjclose
0,000001.SZ,20061229,361.97
1,000002.SZ,20061229,700.45
2,000004.SZ,20061229,16.02
3,000005.SZ,20061229,23.08
4,000006.SZ,20061229,72.25


市值：用于预定义概念矩阵加权

In [4]:
stock2mkt = pd.read_pickle('data2/stock2mkt07_24_date.pkl')
stock2mkt

$market_value
datetime   instrument               
2006-12-29 000001.SZ    2.815605e+06
           000002.SZ    6.747124e+06
           000004.SZ    3.443044e+04
           000005.SZ    2.276691e+05
           000006.SZ    3.400664e+05
...                              ...
2024-05-23 873703.BJ    1.933574e+05
           873706.BJ    9.961285e+04
           873726.BJ    1.554543e+05
           873806.BJ    2.604380e+05
           873833.BJ    7.231208e+04

[19729818 rows x 1 columns]

股票-索引：字典，用于提取batch数值

In [9]:
stock2index = np.load('data2/stock2index._5484.npy', allow_pickle=True)
stock2index.item()['872925.BJ']

0

预定义概念：用于生成预定义概念矩阵

In [10]:
concept_df = pd.read_csv('data2/csi985_concept_all.csv')
concept_df

,end_date,ts_code,bz_item,concept
0,20081231,000001.SZ,个人贷款和垫款,1
1,20081231,000001.SZ,交易性金融资产,1
2,20081231,000001.SZ,代理委托业务,1
3,20081231,000001.SZ,公司贷款和垫款,1
4,20081231,000001.SZ,其他业务,1
...,...,...,...,...
411106,20230630,688799.SH,原料药及中间体,1
411107,20230630,688799.SH,提供服务,1
411108,20230630,688819.SH,其他业务,1
411109,20230630,688981.SH,其他-其他,1


预定义概念矩阵生成过程：大约需要10分钟，生成约1G的文件

In [ ]:
df = concept_df.copy()

# 创建股票和概念的索引字典
stock2ind = stock2index.item() #stock2ind.tolist()

def create_adjacency_matrices(df, stock2index):
    # 提取所有时间点
    times = df['end_date'].unique()
    
    # 初始化结果字典
    adjacency_matrices = {}
    
    for time in times:
        # 过滤出当前时间点的数据
        time_df = df[df['end_date'] == time]
        time_concept2index = {conce: idx for idx, conce in enumerate(time_df['bz_item'].unique())}
        
        # 使用pivot_table创建邻接矩阵
        pivot = time_df.pivot_table(index='ts_code', columns='bz_item', aggfunc='size', fill_value=0)
        
        # 转换为邻接矩阵
        adj_matrix = np.zeros((len(stock2index), len(time_concept2index)))
        for stock in pivot.index:
            for concept in pivot.columns:
                stock_idx = stock2index[stock]
                concept_idx = time_concept2index[concept]
                adj_matrix[stock_idx, concept_idx] = pivot.at[stock, concept]
        
        adjacency_matrices[time] = adj_matrix
    
    return adjacency_matrices

# 调用函数生成邻接矩阵字典
adjacency_matrices2 = create_adjacency_matrices(df, stock2ind)


In [ ]:
# 预定义概念矩阵保存
with open('data2/stock2concept12_6_t_5484.pkl', 'wb') as f:
    pickle.dump(adjacency_matrices2, f)

In [16]:
# 预定义概念矩阵预览
with open('data2/stock2concept12_6_t_5484.pkl','rb') as f:
        stock2concepts=pickle.load(f)

In [20]:
stock2concepts.keys()

dict_keys([20081231, 20091231, 20100630, 20101231, 20110630, 20111231, 20120630, 20121231, 20130630, 20131231, 20140630, 20141231, 20150630, 20151231, 20160630, 20161231, 20170630, 20171231, 20180630, 20181231, 20190630, 20191231, 20200630, 20201231, 20210630, 20211231, 20220630, 20221231, 20230630])

In [23]:
stock2concepts[20091231].shape

(5484, 869)

In [22]:
stock2concepts[20091231]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

以上输入数据中，与日期有关的列尽量都转为datetime格式，stock2concepts例外，其index日期格式为int

训练时 运行 python learn_label.py 即可

## output 结果分析

In [12]:
test_pred = pickle.load(open('output/all_HIST_label11to1_CSRankN/rolling2007-01-01/pred.pkl.test0','rb'))
test_pred

score     label
datetime   instrument                    
2019-01-02 000001.SZ   0.088952  1.322613
           000002.SZ  -0.016623 -0.088791
           000004.SZ  -0.114561 -0.853687
           000005.SZ  -0.068800  1.403080
           000006.SZ  -0.170035 -0.604887
...                         ...       ...
2023-09-28 873339.BJ   0.075396  0.894825
           873527.BJ   0.194868  1.094545
           873576.BJ  -0.111411 -0.501259
           873593.BJ  -0.254025 -1.731561
           873665.BJ  -0.339562 -0.726433

[5021373 rows x 2 columns]

rankIC%

In [14]:
test_pred.groupby(level='datetime').apply(lambda x: x['label'].corr(x['score'], method='spearman')).mean()*100

11.269636199752094

In [15]:
ic_TRA = test_pred.groupby(level='datetime').apply(lambda x: x['label'].corr(x['score'], method='spearman'))
ic_TRA = ic_TRA.reset_index()
ic_TRA.columns = ['date','rankIC']
ic_TRA['y'] = ic_TRA.date.apply(lambda x: str(x)[:4])
ic_TRA.groupby('y').mean(numeric_only=True)*100

,rankIC
y,
2019,13.015870
2020,10.570241
2021,10.562087
2022,12.065687
2023,9.748550
